In [1]:
import numpy as np
from datascience import *
import pandas as pd
import requests

Goal: Country level rankings over time -- and the factors and sensitivities that determine the rankings. 

## Emissions by subsector within each country

In [2]:
countries = requests.get("https://api.climatetrace.org/countries").json()
countries = pd.json_normalize(countries).drop("alpha2", axis=1).rename(columns = {"alpha3":"Abbrev"})
countries = Table.from_df(countries)
countries

name,Abbrev
Aruba,ABW
Afghanistan,AFG
Angola,AGO
Anguilla,AIA
Åland Islands,ALA
Albania,ALB
Andorra,AND
United Arab Emirates,ARE
Argentina,ARG
Armenia,ARM


In [3]:
#emissions data per sector/subsector for all countries from 2015-2020
countries_data = requests.get("https://api.climatetrace.org/emissions_by_subsector_timeseries/AFG?interval=year&since=2010&to=2020").json()
countries_data = pd.json_normalize(countries_data)
for country in countries["Abbrev"][1:]:
    r = requests.get("https://api.climatetrace.org/emissions_by_subsector_timeseries/" + country + "?interval=year&since=2010&to=2020").json()
    country_add = pd.json_normalize(r)
    countries_data = pd.concat([countries_data, country_add], ignore_index = True)

countries_data = Table.from_df(countries_data)
countries_data

country_full,country,sector,subsector,start,end,emissions
Afghanistan,AFG,agriculture,cropland fires,2020-01-01,2021-01-01,156917
Afghanistan,AFG,agriculture,cropland fires,2019-01-01,2020-01-01,22077
Afghanistan,AFG,agriculture,cropland fires,2018-01-01,2019-01-01,53039
Afghanistan,AFG,agriculture,cropland fires,2017-01-01,2018-01-01,27800
Afghanistan,AFG,agriculture,cropland fires,2016-01-01,2017-01-01,26060
Afghanistan,AFG,agriculture,cropland fires,2015-01-01,2016-01-01,25191
Afghanistan,AFG,agriculture,cropland fires,2014-01-01,2015-01-01,nan
Afghanistan,AFG,agriculture,cropland fires,2013-01-01,2014-01-01,nan
Afghanistan,AFG,agriculture,cropland fires,2012-01-01,2013-01-01,nan
Afghanistan,AFG,agriculture,cropland fires,2011-01-01,2012-01-01,nan


In [4]:
agg_countries = requests.get("https://api.climatetrace.org/emissions_by_subsector_timeseries/all?interval=year&since=2010&to=2021").json()
pd.json_normalize(agg_countries)

,country_full,country,sector,subsector,start,end,emissions
0,all,None,agriculture,cropland fires,2020-01-01,2021-01-01,234346742.0
1,all,None,agriculture,cropland fires,2019-01-01,2020-01-01,244224437.0
2,all,None,agriculture,cropland fires,2018-01-01,2019-01-01,208237079.0
3,all,None,agriculture,cropland fires,2017-01-01,2018-01-01,260934782.0
4,all,None,agriculture,cropland fires,2016-01-01,2017-01-01,255406966.0
...,...,...,...,...,...,...,...
424,all,None,waste,wastewater treatment and discharge,2014-01-01,2015-01-01,NaN
425,all,None,waste,wastewater treatment and discharge,2013-01-01,2014-01-01,NaN
426,all,None,waste,wastewater treatment and discharge,2012-01-01,2013-01-01,NaN
427,all,None,waste,wastewater treatment and discharge,2011-01-01,2012-01-01,NaN


## Aggregate emissions by sector within each country

In [5]:
sectors = requests.get("https://api.climatetrace.org/sectors").json()
sectors = pd.json_normalize(sectors)

In [ ]:
emissions = requests.get("https://api.climatetrace.org/emissions_by_country?interval=year&sectors=oil%20and%20gas&since=2010&to=2021").json()
emissions = pd.json_normalize(emissions, record_path = ["emissions"], meta = "country")
emissions["Sector"] = "oil and gas"

for sector in sectors["name"][1:]:
    r = requests.get("https://api.climatetrace.org/emissions_by_country?interval=year&sectors=" + sector + "&since=2010&to=2021").json()
    sector_add = pd.json_normalize(r, record_path = ["emissions"], meta = "country")
    sector_add["Sector"] = sector
    emissions = pd.concat([emissions, sector_add], ignore_index = True)
    
emissions = Table.fromn_df(emissions)